In [1]:
import pandas as pd
import numpy as np
import pprint
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import sem
from scipy.stats import linregress

# Data Load/Setup

In [2]:
# Import CSV files
#Files currently being used
emissions_df = pd.read_csv('Resources/co2_emissions_kt_by_country.csv')
pesticide_df = pd.read_csv('Resources/pesticide-use-tonnes.csv')
population_df = pd.read_csv('Resources/population.csv')

#CSV files from land folder needs cleaning
land_allotment_df = pd.read_csv('Resources/Land/agr_land_percent.csv')
total_land_df = pd.read_csv('Resources/Land/total-land.csv')

#CSV file on crop yields, needs cleaning for ease of use
crop_yields_df = pd.read_csv('Resources/crop_yields.csv')

#CSV files for GDP
agShare_df = pd.read_csv('Resources/agriculture-share-gdp.csv')
gdp_df = pd.read_csv('Resources/national-gdp-penn-world-table.csv')

## Data Cleanup/Modifying


In [3]:
# Preparing emissions, pesticide, population dataframes
emissions_df = emissions_df.rename(columns = {'country_code': 'Code','country_name': 'Country', 'year': 'Year', 'value': 'CO2_Emissions'})

pesticide_df = pesticide_df.rename(columns = {'Entity': 'Country', 
                                              'Pesticides (total) | 00001357 || Agricultural Use | 005157 || Tonnes': 'Total_Pesticides'})

population_df = population_df.rename(columns = {'Entity': 'Country', 'Population (historical)': 'Population'})


In [4]:
#Merging GDP based dataframes
gdp_cleaned_df = pd.merge(gdp_df, agShare_df, how = 'left', left_on = ['Entity', 'Year', 'Code'], right_on = ['Entity', 'Year', 'Code']).dropna()
gdp_cleaned_df = gdp_cleaned_df.rename(columns = {'Entity': 'Country','GDP (output, multiple price benchmarks)': 'GDP',
                                                  'Agriculture, forestry, and fishing, value added (% of GDP)': 'Ag_perc_GDP'})
gdp_cleaned_df['Ag_perc_GDP'] = gdp_cleaned_df['Ag_perc_GDP']/100
gdp_cleaned_df['Agri_GDP'] = gdp_cleaned_df['GDP'] * gdp_cleaned_df['Ag_perc_GDP']

In [5]:
# Modifying agricultural land allotment df
land_holder = land_allotment_df.drop(columns = ['Indicator Name', 'Indicator Code'])


df_long = pd.melt(land_holder, id_vars=['Country Name', 'Country Code'], var_name = 'Year', value_name ='Ag_perc_land')
df_long.rename(columns={'Country Name': 'Country', 'Country Code': 'Code'}, inplace=True)
df_long_sorted_df = df_long.sort_values(by=['Country', 'Year'])
df_long_sorted_df = df_long_sorted_df.reset_index(drop = True)

indicies_to_drop = df_long_sorted_df[df_long_sorted_df['Year'] == 'Unnamed: 68'].index
df_long_sorted_df.drop(index = indicies_to_drop, inplace=True)

df_long_sorted_df['Year'] = df_long_sorted_df['Year'].astype(np.int64)
land_agri_final_df = df_long_sorted_df


In [6]:
#Modifying total_land_df
total_land_df
land_holder = total_land_df.drop(columns = ['Indicator Name', 'Indicator Code'])


df_long = pd.melt(land_holder, id_vars=['Country Name', 'Country Code'], var_name = 'Year', value_name ='Total_Land_sq_km')
df_long.rename(columns={'Country Name': 'Country', 'Country Code': 'Code'}, inplace=True)
df_long_sorted_df = df_long.sort_values(by=['Country', 'Year'])
df_long_sorted_df = df_long_sorted_df.reset_index(drop = True)

indicies_to_drop = df_long_sorted_df[df_long_sorted_df['Year'] == 'Unnamed: 68'].index
df_long_sorted_df.drop(index = indicies_to_drop, inplace=True)

df_long_sorted_df['Year'] = df_long_sorted_df['Year'].astype(np.int64)
total_land_final_df = df_long_sorted_df


In [7]:
#Merging total_land_final_df and land_agri_final_df into land_df
land_df = pd.merge(total_land_final_df, land_agri_final_df, how = 'left', left_on = ['Country', 'Year', 'Code'],
                   right_on = ['Country', 'Year', 'Code']).dropna()
land_df['Agri_Land_sq_km'] = land_df['Total_Land_sq_km'] * land_df['Ag_perc_land']

In [8]:
# Modifying crop_yields_df so that it shows sum crop yield within each country's year
# Note: crop yields are in tonnes per hecatare

test_crop_df = crop_yields_df
all_columns = crop_yields_df.columns.tolist()
crop_columns = all_columns[2:]
crop_yields_df['Total_Crop_Yield_t_ha'] = crop_yields_df[crop_columns].sum(axis=1)

crop_yields_df = test_crop_df.drop(columns=crop_columns)
crop_yields_df = crop_yields_df.rename(columns = {'country' : 'Country', 'year': 'Year'})

In [9]:
#Ensuring all data has the same type of countrys/codes
print(emissions_df['Country'].nunique())
print(pesticide_df['Country'].nunique())
print(population_df['Country'].nunique())
print(gdp_cleaned_df['Country'].nunique())
print(land_df['Country'].nunique())
print(crop_yields_df['Country'].nunique())
print('---------------')
print(emissions_df['Code'].nunique())
print(pesticide_df['Code'].nunique())
print(population_df['Code'].nunique())
print(gdp_cleaned_df['Code'].nunique())
print(land_df['Code'].nunique())

256
251
271
177
258
255
---------------
255
219
250
177
258


In [10]:
#Extracting unique Codes
codes_emi = set(emissions_df['Code'].unique())
codes_pest = set(pesticide_df['Code'].unique())
codes_pop = set(population_df['Code'].unique())
codes_gdp = set(gdp_cleaned_df['Code'].unique())
codes_land = set(land_df['Code'].unique())

#Finding Common Codes
common_codes = codes_emi.intersection(codes_pest).intersection(codes_pop).intersection(codes_gdp).intersection(codes_land)

#Filter DataFrames
emissions_df_filtered = emissions_df[emissions_df['Code'].isin(common_codes)]
pesticide_df_filtered = pesticide_df[pesticide_df['Code'].isin(common_codes)]
population_df_filtered = population_df[population_df['Code'].isin(common_codes)]
gdp_cleaned_df_filtered = gdp_cleaned_df[gdp_cleaned_df['Code'].isin(common_codes)]
land_df_filtered = land_df[land_df['Code'].isin(common_codes)]

#Extracting Common Countries
country_emi = set(emissions_df_filtered['Country'].unique())
country_pest = set(pesticide_df_filtered['Country'].unique())
country_crop = set(crop_yields_df['Country'].unique())

#Finding Common Countries
common_countries = country_emi.intersection(country_pest).intersection(country_crop)

#Filter DataFrames
crop_yields_df_filtered = crop_yields_df[crop_yields_df['Country'].isin(common_countries)]


In [11]:
#Ensuring All Data has the same Country names for their codes
print(emissions_df_filtered['Country'].nunique())
print(pesticide_df_filtered['Country'].nunique())
print(population_df_filtered['Country'].nunique())
print(gdp_cleaned_df_filtered['Country'].nunique())
print(land_df_filtered['Country'].nunique())
print(crop_yields_df_filtered['Country'].nunique())
print('---------------')
print(emissions_df_filtered['Code'].nunique())
print(pesticide_df_filtered['Code'].nunique())
print(population_df_filtered['Code'].nunique())
print(gdp_cleaned_df_filtered['Code'].nunique())
print(land_df_filtered['Code'].nunique())


172
171
171
171
171
147
---------------
171
171
171
171
171


In [12]:
#Merging total_land_final_df and land_agri_final_df into land_df
agriculture_df = pd.merge(land_df_filtered, crop_yields_df_filtered, how = 'left', left_on = ['Country', 'Year'],
                   right_on = ['Country', 'Year']).dropna()

In [13]:
#Extracting all unique Years
years_emi = set(emissions_df_filtered['Year'].unique())
years_pest = set(pesticide_df_filtered['Year'].unique())
years_pop = set(population_df_filtered['Year'].unique())
years_gdp = set(gdp_cleaned_df_filtered['Year'].unique())
years_agri = set(agriculture_df['Year'].unique())

#Finding Common Codes
common_years = years_emi.intersection(years_pest).intersection(years_pop).intersection(years_gdp).intersection(years_agri)

#Filter DataFrames
emissions_final_df = emissions_df_filtered[emissions_df_filtered['Year'].isin(common_years)]
pesticide_final_df = pesticide_df_filtered[pesticide_df_filtered['Year'].isin(common_years)]
population_final_df = population_df_filtered[population_df_filtered['Year'].isin(common_years)]
gdp_final_df = gdp_cleaned_df_filtered[gdp_cleaned_df_filtered['Year'].isin(common_years)]
agriculture_final_df = agriculture_df[agriculture_df['Year'].isin(common_years)]


In [14]:
#Ensuring All Data has the same Country names for their codes
print(emissions_final_df['Country'].nunique())
print(pesticide_final_df['Country'].nunique())
print(population_final_df['Country'].nunique())
print(gdp_final_df['Country'].nunique())
print(agriculture_final_df['Country'].nunique())
print('---------------')
print(emissions_final_df['Code'].nunique())
print(pesticide_final_df['Code'].nunique())
print(population_final_df['Code'].nunique())
print(gdp_final_df['Code'].nunique())
print(agriculture_final_df['Code'].nunique())
print('---------------')
print(emissions_final_df['Year'].nunique())
print(pesticide_final_df['Year'].nunique())
print(population_final_df['Year'].nunique())
print(gdp_final_df['Year'].nunique())
print(agriculture_final_df['Year'].nunique())

171
171
171
171
145
---------------
171
171
171
171
145
---------------
30
30
30
30
30


In [15]:
#Extracting Common Countries
country_emi= set(emissions_final_df['Country'].unique())
country_pest = set(pesticide_final_df['Country'].unique())
country_pop = set(population_final_df['Country'].unique())
country_gdp = set(gdp_final_df['Country'].unique())
country_agri = set(agriculture_final_df['Country'].unique())

#Finding Common Countries
common_countries = country_emi.intersection(country_pest).intersection(country_pop).intersection(country_gdp).intersection(country_agri)

#Filter DataFrames
emi_final_df = emissions_final_df[emissions_final_df['Country'].isin(common_countries)]
pest_final_df = pesticide_final_df[pesticide_final_df['Country'].isin(common_countries)]
pop_final_df = population_final_df[population_final_df['Country'].isin(common_countries)]
gdpA_final_df = gdp_final_df[gdp_final_df['Country'].isin(common_countries)]
agri_final_df = agriculture_final_df[agriculture_final_df['Country'].isin(common_countries)]

In [16]:
#Ensuring All Data has the same Country names for their codes
print(emi_final_df['Country'].nunique())
print(pest_final_df['Country'].nunique())
print(pop_final_df['Country'].nunique())
print(gdpA_final_df['Country'].nunique())
print(agri_final_df['Country'].nunique())
print('---------------')
print(emi_final_df['Code'].nunique())
print(pest_final_df['Code'].nunique())
print(pop_final_df['Code'].nunique())
print(gdpA_final_df['Code'].nunique())
print(agri_final_df['Code'].nunique())
print('---------------')
print(emi_final_df['Year'].nunique())
print(pest_final_df['Year'].nunique())
print(pop_final_df['Year'].nunique())
print(gdpA_final_df['Year'].nunique())
print(agri_final_df['Year'].nunique())

145
145
145
145
145
---------------
145
145
145
145
145
---------------
30
30
30
30
30


In [17]:
emi_final_df= emi_final_df.sort_values(by=['Country', 'Year'])


# Exporting Cleaned Data Placement in MongoDB


In [18]:
#Exporting CSV files

emi_final_df.to_csv('emissions.csv',index=False)
pest_final_df.to_csv('pesticides.csv',index=False)
pop_final_df.to_csv('population.csv',index=False)
gdpA_final_df.to_csv('gdp.csv',index=False)
agri_final_df.to_csv('agriculture.csv',index=False)
